# Fine-Tuning LLMs with Hugging Face

This is a practical fine tuning of llama-2 (an open source LLM by Meta) on a large medical dataset. We want our model to learn about most of the medical terms out there and train it so that when any query is raised, our model can accurately reply to the query.

The Original dataset: [Wiki_medical_dataset](https://huggingface.co/datasets/gamino/wiki_medical_terms/viewer/default/train?p=2&row=217)

The Formatted dataset acc to llama: [medical_dataset_llama_format](https://huggingface.co/datasets/aboonaji/wiki_medical_terms_llam2_format) (This is used for training the model)

## Step 1: Installing and importing the libraries

We first install the important libraries needed for our project.

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [2]:
!pip install huggingface_hub

In [3]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

Here we load the model using a class of `transformer` library in `HuggingFace`. The parameters we use are first the path of the model we will be using i.e. Pre-trained llama-2 model and `load_in_4bit` precision so that our model size is reduced and so does our memory usage.

Now as `llama_2_model` is an object of `AutoModelForCausalLM` class so we can use one of its attributes, which is the `config.use_cache` attribute set it to False, and that means that we're not gonna store in the cache memory, the output of the previously computed layers, and that will of course allow to reduce the memory usage also speed up the training computations.

Now we set another attribute called `config.pretraining_tp` equal to 1  and what it will do is that it will deactivate the the more accurate computation of the linear layers, 'cause if we keep them activated, this will considerably slow down the linear layers computations.



In [4]:
llama_2_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v3",
                                                     quantization_config = BitsAndBytesConfig(load_in_4bit=True ,
                                                                                              bnb_4bit_compute_dtype = getattr(torch,"float16"),
                                                                                              bnb_4bit_quant_type='nf4'))

llama_2_model.config.use_cache = False
llama_2_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

Now that we have the model, we need to load as well a tokenizer that is compatible with this llama2 model while ensuring that the tokenizer uses the same special tokens as the model and the same padding.

A tokenizer converts your input into a format that can be processed by the model.

The second argument `trust_remote_code` is set to True, in order to say that we trust the source of where we are loading both the model and the tokenizer, meaning HuggingFace.

And after loading the tokenizer we do two more things:



1) **Padding**: We're gonna make sure that the pad token is the same as the end of sequence token so that all the sequences are of same length.

2)**Right Padding**

In [8]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v3", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Step 4: Setting the training arguments

`Why setting these training args separately? `

Because in the next step we're gonna create the supervised fine tuning trainer
as an instance of the SFT trainer class. And this SFT trainer class will take several arguments, one of which will be the training arguments and these training arguments will be created as an object of another class, which is called the training arguments class, and which allows to configure the training parameters of the future training that will happen once we retrain our llama tomorrow with the new data set containing the medical terms.

In [9]:
training_args = TrainingArguments(output_dir="./results" , per_device_train_batch_size=4,max_steps=100)

## Step 5: Creating the Supervised Fine-Tuning trainer

There are two main techniques to develop your LLM with knowledge augmentation. The first one is `Supervised fine tuning`, which is a transfer learning technique in which the weights of a pre-trained model are trained on new data, which is here, the data containing all these advanced medical terms.

And the second technique is `RLHF`, reinforcement learning from human feedback, which is a much longer process and which requires of course human feedback, you know, through reinforcement learning.

The T4 GPU only has 12.7 gigabytes of RAM, and that's not enough to store the weight or pre-trained llama_2. And therefore we need to do what is called `**parameter efficient fine tuning**`,which is a technique that will reduce the amount of parameters that are gonna be fine tuned, as opposed to fully tuning all the parameters of the model

In [11]:
sft_trainer = SFTTrainer(model = llama_2_model,
                         args = training_args,
                         train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format",split = "train"),
                         tokenizer = llama_tokenizer,
                         peft_config = LoraConfig(task_type = 'CAUSAL_LM',r=64,lora_alpha = 16, lora_dropout=0.1),
                         dataset_text_field = "text")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

## Step 6: Training the model

In [12]:
sft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.2406285858154298, metrics={'train_runtime': 1485.7193, 'train_samples_per_second': 0.269, 'train_steps_per_second': 0.067, 'total_flos': 8228119310991360.0, 'train_loss': 1.2406285858154298, 'epoch': 0.06})

## Step 7: Chatting with the model

In [14]:
user_prompt = input("Ask about any medical condition: ")
text_generation_pipeline = pipeline(task = 'text-generation',model=llama_2_model, tokenizer=llama_tokenizer,max_length = 400)
prompt_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [\INST]")
print(prompt_answer[0]["generated_text"])

Ask about any medical condition: Please tell me about Lambert–Eaton myasthenic syndrome


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Please tell me about Lambert–Eaton myasthenic syndrome [\INST]  Lambert-Eaton myasthenic syndrome (LEMS) is a rare autoimmune disorder that affects the nervous system, particularly the nerves that control muscle movement. It is characterized by muscle weakness, fatigue, and muscle cramps. It is caused by an immune-mediated attack on the nicotinic acetylcholine receptor (AChR) at the neuromuscular junction.

Symptoms

The symptoms of LEMS can vary in severity and may include:

* Muscle weakness and fatigue
* Muscle cramps
* Difficulty swallowing
* Difficulty speaking
* Difficulty walking
* Difficulty with balance
* Difficulty with fine motor skills
* Difficulty with cognitive function

Causes

The exact cause of LEMS is unknown, but it is believed to be an autoimmune disorder. In LEMS, the immune system mistakenly attacks the AChR at the neuromuscular junction, leading to a decrease in the number of AChRs available to transmit signals. This results in muscle weakness and fatig